In [1]:
# importing the required libraries
import numpy as np
from statistics import mode


The following function splits the data into the rows that go to the left and right child based on the condition. If c_id is a categorical variable, then euqality condition is checked. If c_id is a numerical variable, then less than condition is checked. "Yes" condition goes to the left child and "No" goes to the right child.

In [2]:
def split_data(c_id, split_value, data, schema):
    cat_cols, num_cols = schema
    if c_id in num_cols:
        left_split = [row for row in data if row[c_id] < split_value]
        right_split = [row for row in data if row[c_id] >= split_value]
    else:
        left_split = [row for row in data if row[c_id] == split_value]
        right_split = [row for row in data if row[c_id] != split_value]
    return left_split, right_split


The following two functions, as the name suggests, are to calculate the entropy and information gain, respectively, for each split.

In [3]:
def entropy(data, labels):
    
    total_size = len(data)
    if total_size==0:
        return(0)
    entropy = 0
    for label in labels:
        label_count = len([row for row in data if row[-1] == label]) # number of rows with target=label
        if label_count == 0:
            continue
        entropy += label_count * np.log(label_count/total_size) # weighted sum
    entropy = -entropy/total_size # averaging
    return(entropy)

def information_gain(left_split, right_split, labels):
    
    total_size = len(left_split) + len(right_split)
    if total_size==0: # if no more data, then stop
        return(0)
    total_data = left_split + right_split
    parent_entropy = entropy(total_data, labels) # parent's entropy
    left_entropy = entropy(left_split, labels) # left child's entropy
    right_entropy = entropy(right_split, labels) # right child's entropy
    ie = parent_entropy - (len(left_split)/total_size)*left_entropy - (len(right_split)/total_size)*right_entropy
    return(ie)
 


The following function determines the best decision based on the information gain. 

In [4]:
def get_decision_node(data, num_predictors, labels, schema):
    node_options = dict()
    for c_id in range(num_predictors):
        for row in data:
            left_split, right_split = split_data(c_id, row[c_id], data, schema) # split data based on the condition
            ie = information_gain(left_split, right_split, labels) # fetch the information gain
            node_options[(c_id, row[c_id])] = (ie, left_split, right_split) # storing
    if len(node_options) != 0:
        # choosing the best decision option (maximum information gain)
        (best_c_id, best_split_value), (ie, left_split, right_split) = sorted(node_options.items(), key=lambda item: item[1][0])[-1]
        return ({'c_id': best_c_id, 'split_value': best_split_value, 'left_split': left_split, 'right_split': right_split})         
            

The following function splits the root node further into child nodes. It builds the entire tree. It also takes into consideration the max_depth and min_leaf_size parameters.

In [5]:
def split_node(node, node_depth, max_depth, min_leaf_size, num_predictors, labels, schema):
    
    left_split = node['left_split']
    right_split = node['right_split']
    
    if (left_split == []) or (right_split == []): # leaf node
        node['left_child'] = mode([row[-1] for row in left_split+right_split])
        node['right_child'] = mode([row[-1] for row in left_split+right_split])
        return
    
    if node_depth >= max_depth: # depth limit exceeded, end the split; hence leaf node
        node['left_child'] = mode([row[-1] for row in left_split])
        node['right_child'] = mode([row[-1] for row in right_split])
        return
    
    if len(left_split) <= min_leaf_size: # size limit exceeded, end the split; hence leaf node
        node['left_child'] = mode([row[-1] for row in left_split])
    else: # continue with the left child
        node['left_child'] = get_decision_node(left_split, num_predictors, labels, schema)
        split_node(node['left_child'], node_depth+1, max_depth, min_leaf_size, num_predictors, labels, schema)
        
    if len(right_split) <= min_leaf_size: # size limit exceeded, end the split; hence leaf node
        node['right_child'] = mode([row[-1] for row in right_split])
    else: # continue with the right child
        node['right_child'] = get_decision_node(right_split, num_predictors, labels, schema)
        split_node(node['right_child'], node_depth+1, max_depth, min_leaf_size, num_predictors, labels, schema)  


The following function builds individual trees, given a dataset, max_depth and min_leaf_size.

In [6]:
def train_tree(data, max_depth, min_leaf_size):
    labels = np.unique([row[-1] for row in data]) # 
    num_predictors = len(data[0])-1
    
    cat_cols = []
    num_cols = []
    for c in range(len(data[0][:-1])):
        if isinstance(data[0][c], str):
            cat_cols.append(c)
        else:
            num_cols.append(c)
    
    schema = (cat_cols, num_cols)
    
    tree = get_decision_node(data, num_predictors, labels, schema) # getting the root node
    
    # splitting the root node further and building the tree
    split_node(node=tree, node_depth=1, max_depth=max_depth, min_leaf_size=min_leaf_size, 
               num_predictors=num_predictors, labels=labels, schema=schema)
    
    return(tree, schema)

The following function builds a random forest by building individual trees from the above functions. 
* sample_ratio --> The ratio of data to be choses as subset data (for each tree)
* n_features --> number of features to be used in a subset (for each tree)

In [7]:
def random_forest(X_train, n_estimators, max_depth, min_leaf_size, sample_ratio, n_features):
    
    random_forest = []
    cols_count = len(X_train[0])-1
    for i in range(n_estimators):
        # subsetting rows with replacement
        data_subset_index = np.random.randint(0, len(X_train), round(len(X_train)*sample_ratio))
         # subsetting columns without replacement
        col_subset_index = sorted(np.random.permutation(cols_count)[:n_features])
        data_subset = []
        for row_id in data_subset_index: # subsetted rows
            old_row = X_train[row_id] # full row
            new_row = [old_row[col_id] for col_id in col_subset_index] # subsetted features
            new_row.append(old_row[-1]) # adding target value
            data_subset.append(new_row)
        tree, schema = train_tree(data_subset, max_depth, min_leaf_size) # building a shallow tree with the subsetted data
        random_forest.append([col_subset_index, tree, schema])
    
    return(random_forest)


The following are functions to make predictions, given a dataset and a random forest.

In [8]:
# predict for a single row
def predict_row(row, tree, schema):
    cat_cols, num_cols = schema
    if tree['c_id'] in num_cols: # if a numerical column
        if row[tree['c_id']] < tree['split_value']:
            if isinstance(tree['left_child'], dict): # if further split exists
                y_pred = predict_row(row, tree['left_child'], schema)
            else: 
                y_pred = tree['left_child']
        else:
            if isinstance(tree['right_child'], dict): # if further split exists
                y_pred = predict_row(row, tree['right_child'], schema)
            else:
                y_pred = tree['right_child']
    else:  # if a categorical column
        if row[tree['c_id']] == tree['split_value']: 
            if isinstance(tree['left_child'], dict): # if further split exists
                y_pred = predict_row(row, tree['left_child'], schema)
            else:
                y_pred = tree['left_child']
        else:
            if isinstance(tree['right_child'], dict): # if further split exists
                y_pred = predict_row(row, tree['right_child'], schema)
            else:
                y_pred = tree['right_child']
    return(y_pred)


def predict(data, rf):
    accuracy = 0
    for row in data:
        y_true = row[-1]
        tree_predictions = []
        for tree_info in rf:
            col_subset_index = tree_info[0]
            tree = tree_info[1] 
            schema = tree_info[2]
            data_row = [row[col_id] for col_id in col_subset_index] # subsetted features
            tree_pred = predict_row(data_row, tree, schema)
            tree_predictions.append(tree_pred)
        y_pred = mode(tree_predictions)
        print("Individual tree predictions: ", tree_predictions)
        print("True Label: ", y_true, " Predicted Label: ", y_pred)
        if y_true==y_pred:
            accuracy += 1
    accuracy = accuracy/len(data)
    print("Accuracy: ", accuracy)
    

In [9]:
# toy data

X_train = [['Y', 'Y', 7, 0],
      ['Y', 'N', 12, 0],
      ['N', 'Y', 18, 1],
      ['N', 'Y', 35, 1],
      ['Y', 'Y', 38, 1],
      ['Y', 'N', 50, 0],
      ['N', 'N', 83, 0]
     ]
X_test = [['Y', 'Y', 8, 0],
      ['Y', 'Y', 12.5, 0],
      ['N', 'N', 20, 1]]

In [19]:
# building a random forest
rf = random_forest(X_train, n_estimators=5, max_depth=1, min_leaf_size=1, sample_ratio=1, n_features=2)


In [20]:
predict(X_train, rf) # training metrics


Individual tree predictions:  [1, 1, 0, 0, 0]
True Label:  0  Predicted Label:  0
Individual tree predictions:  [0, 0, 0, 0, 0]
True Label:  0  Predicted Label:  0
Individual tree predictions:  [1, 1, 1, 1, 0]
True Label:  1  Predicted Label:  1
Individual tree predictions:  [1, 1, 1, 1, 1]
True Label:  1  Predicted Label:  1
Individual tree predictions:  [1, 1, 1, 0, 1]
True Label:  1  Predicted Label:  1
Individual tree predictions:  [0, 0, 1, 0, 1]
True Label:  0  Predicted Label:  0
Individual tree predictions:  [0, 0, 1, 1, 1]
True Label:  0  Predicted Label:  1
Accuracy:  0.8571428571428571


In [21]:
predict(X_test, rf) # test metrics


Individual tree predictions:  [1, 1, 0, 0, 0]
True Label:  0  Predicted Label:  0
Individual tree predictions:  [1, 1, 0, 0, 0]
True Label:  0  Predicted Label:  0
Individual tree predictions:  [0, 0, 1, 1, 0]
True Label:  1  Predicted Label:  0
Accuracy:  0.6666666666666666
